In [1]:
# Import the necessary dependencies

# Operating System
import os

# Numpy, Pandas and Scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix, save_npz, load_npz

# Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Model Evaluation
from evaluation import evaluate

# BLU12 - Learning Notebook - Workflow
## Introduction
This week we are going to simulate the real-life environment of the Hackathon! We will provide you with a dataset, some tips and you are going to train, validate and predict with a Recommender System using all the knowledge acquired throughout this specialization. This exercise is very similar to the Hackathon of the previous year.

## Context
You have finished the Academy and you are hired as a Data Scientist in Recommentunify *(ignore the horrible company name, it works and we have more important stuff to discuss)*.

First week and the job and the CEO of the next-unicorn-startup that hired you pitches your first task for the company. NASA has discovered that "Despacito 2" is on the making and you need to **urgently avoid this scenario.**

<img src="./media/despacito_nasa.jpg" alt="Miss me yet?" width="500"/>

He doesn't care how you do it or how it works, just that you use fancy trendy eye-catching mind-blowing AI *stuff* to promote Recommentunify's name in the Data Science industry.

## Technical Task
After the pitch from the CEO, you talk with your Lead Data Scientist and you get the tasks needed to fulfil the job. The company collected some data on the users, you will need to create the Recommender System that is effective enough so that your users will not need a 2nd version of a Despacito.

Your Lead DS keeps some of the data as a test set (in which you will estimate production performance) and gives you the remaining data for yourself.

### Technical Details - Type Hints
Since this specialization focuses on the workflow, we will add a cool new feature of more recent versions of Python called "Type Hints". Hints allow you to document the code "within itself" and allow for new users and reviewers to better understand what you are doing (consider your future self as a new user when you have to pick up code that you've created a long time ago). This is basically adding the type of the argument after the argument name in the function. You can also hint what should be the return type. If you do not use anything else, Python will not enforce the argument type (will acept as it is), but this is still useful for documentation and debugging.

```
def add_two(x: int) -> int: return x+2

print(add_two(2))     # receives int and returns 4, which is an int
print(add_two(2.2))   # receives float and returns 4.2 , which is also float (still works)
```

### Technical Details - Docstrings
You can make use of the [Docstrings](https://en.wikipedia.org/wiki/Docstring) to better document your code. There are several styles available. We will use the "Google Style" for this notebook.

As an example, you can document in a function its overall behaviour (what is used for), what are the expected parameters and returns (along with their types), as well as defining the TODOs for the difficult tasks you are too lazy to code yourself *(future self will be wiser to pick-up on this).*

## Step -1: Understanding the data
You have available under the `./data/` folder some files that you can use for building the Recommender System. As a good scientist, the first step is to validate that the data you have will be good enough to proceed.

The files we have available for the **training** stage are:
* `train_play_counts.txt` has the listening history for each user
* `song_tag.csv`: has the relationship between tags and songs
* `songs.txt`: has the correspondence between the song_id (a unique id, eg: SOAAADD12AB018A9DD) and the song_index (an integer index, eg: 1)
* `tags.csv`: has the correspondence between the tag_id (a tag name, eg: classic rock) and the tag_index (an integer index, eg: 1)



For the **test** stage, we have:
* `test_users.csv`: has the user_ids for which a recommendation should be produced
* `example_output.csv`: has an example of an output (just for a couple of users)

The best approach is to look at the raw files and print out the first rows of each file just to get a overall sense of what we have.

*Note for Windows users: remember that you need to convert the head command into the Windows equivalent. Tip: you can use what you learned in Data Wrangling Learning Unit!*

In [2]:
print("train_play_counts.txt \n")
!head -3 data/train_play_counts.txt

train_play_counts.txt 



'head' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
print("song_tag.csv\n")
!head -4 data/song_tag.csv

song_tag.csv



'head' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
print("songs.txt \n")
!head -3 data/songs.txt

songs.txt 



'head' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
print("tags.csv \n")
!head -4 data/tags.csv

tags.csv 



'head' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
print("test_users.csv \n")
!head -3 data/test_users.csv

test_users.csv 



'head' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
print("example_output.csv \n")
!head -1 data/example_output.csv

example_output.csv 



'head' is not recognized as an internal or external command,
operable program or batch file.


## Step 0: Load the Data
After validating that the data we have is good enough, we start with building the ratings matrix. Our strategy for the first model is to get a non-personalized system as a baseline. This allows us to both predict for existing users (no cold-start problem) and for new users.

Note: since the `song_tag.csv` is heavy, it was stored in a zip file which you will need to unzip.

## Step 0.1: Load the Train and Test files

In [8]:
def read_users_history() -> pd.DataFrame:
    """Imports the listening history for each user.
    
    Returns:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
    """
    path = os.path.join('data', 'train_play_counts.txt')
    data = pd.read_csv(path, names=['user', 'song_id', 'rating'], sep='\t')
    return data

data = read_users_history()
data.head()

,user,song_id,rating
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
user       100000 non-null object
song_id    100000 non-null object
rating     100000 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [10]:
data.user.unique().shape

(7526,)

In [11]:
# How many ratings do we have in total?
# Tip: The ":," at the end of the f-string adds the thousand separator.
print(f"We have {len(data):,} ratings in total.")

We have 100,000 ratings in total.


In [12]:
def read_test_users() -> pd.DataFrame:
    """Imports the list of users for which we need to predict.
    
    Returns:
        users_to_pred (pd.DataFrame): DataFrame with the users for which we will recommend songs. 
    """

    path = os.path.join('data', 'test_users.csv')
    users_to_pred_ = pd.read_csv(path, names=['users to recommend songs'])
    
    return users_to_pred_


users_to_pred = read_test_users()
users_to_pred.head()

,users to recommend songs
0,56d985c92960b98ad76a48b10a062b0cd86795bf
1,0102d1549242d159df98f333aee4041f96f37e98
2,991411f0dca94f348c7bd3eae93b6e6c061605f1
3,03feb8ee0424fc5c0bafb25e4bdc6d9380a3caba
4,323fbb28144eefa3eabfa22bd310dfb0713de80d


In [13]:
# For how many users are we recommending stuff?
print(f"We have {len(users_to_pred)} users in need for better songs!")

We have 1000 users in need for better songs!


## Step 0.2: Compare the train and test files

In [14]:
# And for how many users we already know something?
# For the 1000 users in our test set, we have data for 700 in the original listening history.
# For the remaining 300 we will have to use non-personalized strategies.
users_to_pred.isin(data.user.values).sum()

users to recommend songs    700
dtype: int64

In [15]:
def get_indices_from_users_to_pred(users_to_pred: pd.DataFrame, data: pd.DataFrame):
    """Get the indices of users_to_pred for which we have data and for which we don't.
    
    Args:
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        data (pd.DataFrame): Original of listening history for the users.
        
    Returns:
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
        index_users_not_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's without training data.
        
    """
    index_users_in_data = users_to_pred[users_to_pred.isin(data.user.values).values].index
    index_users_not_in_data = users_to_pred[~users_to_pred.isin(data.user.values).values].index
    
    return index_users_in_data, index_users_not_in_data


index_users_in_data, index_users_not_in_data = get_indices_from_users_to_pred(users_to_pred, data)

In [16]:
# For further inspection, we advise you to look at the objects themselves.
print(f"The index for users which we have training data has length of {len(index_users_in_data)}.")
print(f"The index for users which we don't have training data has length of {len(index_users_not_in_data)}.")

The index for users which we have training data has length of 700.
The index for users which we don't have training data has length of 300.


In [17]:
def get_users_to_pred_by_index(users_to_pred, index_users_in_data):
    """DataFrame with user_id's in test set for for which we have training data.

    Args: 
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
    Returns:
        users_in_data (pd.DataFrame): Dataframe containing the list of user_id's with training data.
    
    """
    return users_to_pred.iloc[index_users_in_data].reset_index(drop=True)

# Get the test users with training data
test_users_in_data = get_users_to_pred_by_index(users_to_pred, index_users_in_data)
print(test_users_in_data.shape)
test_users_in_data.head()

(700, 1)


,users to recommend songs
0,56d985c92960b98ad76a48b10a062b0cd86795bf
1,991411f0dca94f348c7bd3eae93b6e6c061605f1
2,323fbb28144eefa3eabfa22bd310dfb0713de80d
3,55c750f0951ca1021b26c0e758660bb8a2c49d3a
4,b458e3d697276a93aa6926caf1ff08e875933940


In [18]:
# Get the test users without training data
test_users_not_in_data = get_users_to_pred_by_index(users_to_pred, index_users_not_in_data)
print(test_users_not_in_data.shape)
test_users_not_in_data.head()

(300, 1)


,users to recommend songs
0,0102d1549242d159df98f333aee4041f96f37e98
1,03feb8ee0424fc5c0bafb25e4bdc6d9380a3caba
2,02e4174f754ee4546f4c4438a4bea6ed877c6c4f
3,03070a3284db565b2e67f8ad01ce96c31c9986dd
4,01dd0b2bb39643b8925a672105819d599f827c87


## Step 0.3: Create the Ratings matrix

In [19]:
def make_ratings(data: pd.DataFrame) -> csr_matrix:
    """Creates the ratings matrix of listening history.
    
    Creates the ratings matrix from the listening history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        ratings (csr_matrix): ratings matrix with shape (n_users, n_items)
        
    TODO:
        * Add the shape as an optional parameter.
        
    """
    # You have probably seen this before
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape).tocsr()
    
    return R_


R = make_ratings(data)
R

<7526x41194 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [20]:
data_pivot = data.pivot(index='user', columns='song_id', values='rating').sort_index().fillna(0)
data_pivot_csr = csr_matrix(data_pivot)

In [21]:
data_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7526 entries, 0011d5f4fb02ff276763d385c3f2ded2b00ad94a to ffe98099df91d5ff4a9ab02f58bb9dc87b278cb2
Columns: 41194 entries, SOAAAGQ12A8C1420C8 to SOZZZRV12A8C1361F1
dtypes: float64(41194)
memory usage: 2.3+ GB


In [22]:
(data_pivot_csr != R).todense().sum()

0

In [23]:
# Just for mental (in)sanity, let's match the info of the matrix to what is printed in the previous cell.
print(f"The shape is {R.shape}")
print(f"The dtypes of the elements are {R.dtype}")
print(f"The number of stored elements is {R.nnz}")
print(f"The type of the matrix is {type(R)}")

The shape is (7526, 41194)
The dtypes of the elements are int64
The number of stored elements is 100000
The type of the matrix is <class 'scipy.sparse.csr.csr_matrix'>


In [24]:
# Let's store a Series with the unique user id's that we have in the original data.
def get_unique_users(data):
    """Get unique users in training data.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        unique_users (pd.DataFrame): DataFrame of one column with unique users in training data.
    
    """
    return pd.DataFrame(np.unique(data.iloc[:, 0].values), columns=["users to recommend songs"])


unique_users_training_data = get_unique_users(data)
unique_users_training_data.head()

,users to recommend songs
0,0011d5f4fb02ff276763d385c3f2ded2b00ad94a
1,002511b392561fc1d426d875c386b356a6fc5702
2,002dfbc3c073b55a64a4abab34c0ca1f13897f1c
3,003998bc33cddeba02428a43391c6716e523c8f7
4,0042d2027dfa0340e31d2aa875c4be229730efb7


In [78]:
(np.sort(unique_users_training_data.values.reshape(-1,)) != np.sort(data.user.unique())).sum()

0

## Step 0.4: Record the indices of users
After having the users for the test set and the original ratings matrix, let's create some indices which we will use later on to filter data. Don't worry if it does not seem much intuitive in the beginning, you will further return to this and get a better grasp.

In [25]:
def get_indices_in_ratings_for_test_users_in_data(data: pd.DataFrame, test_users_in_data: pd.DataFrame):
    """Returns the index of the ratings matrix for the test users for which we have training data.
    
    Args:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
        test_users_in_data (pd.DataFrame): DataFrame containing the list of test users for which we have training data.
        
    Returns:
        indices_ratings_tests_users_in_data (np.array): Indices of users in test set with training data for ratings matrix.
    
    """
    unique_users = get_unique_users(data).iloc[:, 0]
    # df.index.to_numpy() not available before pandas version 0.24.0
    #indices_ratings_tests_users_in_data = unique_users[unique_users.isin(test_users_in_data.iloc[:, 0])].index.to_numpy()
    indices_ratings_tests_users_in_data = unique_users[unique_users.isin(test_users_in_data.iloc[:, 0])].index.values
    return indices_ratings_tests_users_in_data

indices_ratings_tests_users_in_data = get_indices_in_ratings_for_test_users_in_data(data, test_users_in_data)

In [26]:
len(indices_ratings_tests_users_in_data)

700

In [27]:
print(f"As expected, the length of the indices should be {len(indices_ratings_tests_users_in_data)}, matching the number of users in test set with training data.")

As expected, the length of the indices should be 700, matching the number of users in test set with training data.


## Step 1: Understand your data flow
In the end, we will be predicting over a test set of users, which may or may not have ratings available in our training data. For the ones in the test set for which we have data in the training, we may use personalized recommendation systems. Otherwise, we need to switch to non-personalized recommendations. 

To split into train and validation, we undersample the original list of ratings but we need to make sure that the overall shape is the same (making sure that by undersampling we do not affect the shape of the full matrix).

In [28]:
def make_ratings(data: pd.DataFrame, shape: tuple = None) -> csr_matrix:
    """Creates the ratings matrix of listening history with optional shape
    
    2nd version of the make_ratings which handles the previous TODO.
    Creates the ratings matrix from the listening history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  Listening history for the users.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from data argument.
        
    Returns:
        ratings (csr_matrix): Ratings matrix with shape (n_users, n_items).
    
    """
    # You have probably seen this before
    users, user_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    items, item_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    if shape == None:
        shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape).tocsr()
    return R_


R = make_ratings(data)
R

<7526x41194 sparse matrix of type '<class 'numpy.int64'>'
	with 100000 stored elements in Compressed Sparse Row format>

In [115]:
data_test1 = data.sample(frac=0.2, random_state=12).sort_index()

In [95]:
data.sample(frac=0.2, random_state=12).index.values

array([23441, 90125, 57892, ..., 35701, 95879, 12911], dtype=int64)

In [122]:
sample_idx_values = data.sample(frac=0.2, random_state=12).index.values
sample_idx_bool = data.index.isin(sample_idx_values)
not_sample_idx_bool = ~data.index.isin(sample_idx_values)

In [116]:
data_test2 = data.loc[data.sample(frac=0.2, random_state=12).index.values,:].sort_index()

In [118]:
data_test3 = data[sample_idx_bool].sort_index()

In [117]:
(data_test1 != data_test2).sum()

user       0
song_id    0
rating     0
dtype: int64

In [119]:
(data_test1 != data_test3).sum()

user       0
song_id    0
rating     0
dtype: int64

In [29]:
# Percentage of listening history used for validation.
test_size = 0.2

def make_train_val_split(data: pd.DataFrame, test_size : float = 0.2, shape: tuple = None):
    """Split the data into train and validation and returns the ratings matrixes accordingly.
    
    Args:
        data (pd.DataFrame): Listening history for the users.
        test_size (float): Percentage of listening history used for validation.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from 'data' argument.
    
    Returns:
        ratings_train (csr_matrix): Ratings matrix for train.
        ratings_val (csr_matrix): Ratings matrix for validation.
    
    """
    train_data, val_data = train_test_split(data, test_size=test_size)
    R_train = make_ratings(train_data, shape=shape)
    R_val = make_ratings(val_data, shape=shape)
    return R_train, R_val

ratings_train, ratings_val = make_train_val_split(data, test_size=test_size, shape=R.shape)

In [30]:
# After the train/validation split, let's compare the number of ratings available in each matrix.
print(f"After the split we have {ratings_train.nnz:,} ratings in the train set and {ratings_val.nnz:,} ratings in the validation set.")

After the split we have 80,000 ratings in the train set and 20,000 ratings in the validation set.


## Step 2: Non-Personalized
Let's build our baseline using one of the the Non-Personalized techniques we learned in BLU10 - "Popular Items". This basically fetches the items which have more ratings overall (we don't care if it is good or bad). Is this how it should be?

<img src="./media/fry_good_bad.jpg" alt="Confused Fry" width="300"/>


Probably not. But it is worth a shot!

In [31]:
def get_most_rated(ratings: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items in a ratings matrix.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    
    def count_ratings(R_):
        return np.array(R_.sum(axis=0))[0]
    
    ratings_ = count_ratings(ratings)
    return np.negative(ratings_).argsort()[:n]


non_pers_most_rated = get_most_rated(ratings_train, 100)
non_pers_most_rated

array([ 2291,  1189, 26658,  8341,  1337,  6291, 29451, 20000,  2308,
        4573, 27842, 28289, 22541, 17599, 19514, 28702,  5007, 11369,
       22463, 16195, 16235, 26894, 33217, 30780, 30378, 28185,  7985,
       22489, 16225,  1771, 18976, 20054, 28933, 24625, 29084, 13069,
        5095, 34519, 20412, 14408, 22979,  4822, 32433,  6790, 22308,
       30691, 12300, 16823, 15149, 13078, 27322,  8099, 32125, 24154,
       26752, 15837,  9679, 30600, 27807,  3606, 20365,  2263, 25243,
       25546, 19506, 35474, 28284,  4575, 18821, 17392, 14076, 28591,
       34621, 32616, 16509, 19395, 15597, 17920, 24603, 23191, 22718,
       28661, 27860, 19068,   191, 32990, 19132,  4201, 18527, 17237,
       27806, 17362,   793, 17111,  4794, 32703,  7903,  5752, 28148,
       22642], dtype=int64)

In [32]:
def convert_non_pers_recommendations_to_df(non_pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """
    Converts the non-personalized most rated to an DataFrame with the users and the recommendations.
    We will basically repeat the non_pers_recs array for the number of users in need.
    
    Args:
        non_pers_recs (np.array): Array of indices for the best non-personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    non_pers_df = pd.DataFrame(np.zeros((len(users_to_pred), 1), dtype=non_pers_recs.dtype) + non_pers_recs)
    non_pers_df = pd.concat([users_to_pred, non_pers_df], axis=1)
    non_pers_df = non_pers_df.set_index("users to recommend songs")
    
    return non_pers_df


non_pers_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, unique_users_training_data)
non_pers_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
002511b392561fc1d426d875c386b356a6fc5702,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
003998bc33cddeba02428a43391c6716e523c8f7,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
0042d2027dfa0340e31d2aa875c4be229730efb7,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642


In [173]:
one_user_pred_df = pd.DataFrame(non_pers_most_rated).T
number_of_users = unique_users_training_data.shape[0]

preds_withoud_users_df = pd.concat([one_user_pred_df]*number_of_users, axis=0, ignore_index=True)
preds_withoud_users_df = preds_withoud_users_df.reset_index(drop=True)

non_pers_most_rated_df2 = pd.concat([preds_withoud_users_df, unique_users_training_data.reset_index(drop=True)], axis=1)
non_pers_most_rated_df2 = non_pers_most_rated_df2.set_index('users to recommend songs', drop=True)

#non_pers_most_rated_df2

In [174]:
non_pers_most_rated_df.equals(non_pers_most_rated_df)

True

In [33]:
def create_dict_preds(preds_df: pd.DataFrame) -> dict:
    """Convert the predictions DataFrame (index:users -> columns: items) to a dictionary of key (user->list of items).
    
    Args: 
        preds_df (pd.DataFrame): DataFrame containing the users and the ordered predictions.
        
    Returns:
        preds_dict (dict): Dict of (user_id: list of items) used for evaluating the performance.
    
    """
    return {preds_df.index[i]: preds_df.values[i].tolist() for i in range(len(preds_df))}


non_pers_dict = create_dict_preds(non_pers_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(non_pers_dict.items())[0:1])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [2291,
  1189,
  26658,
  8341,
  1337,
  6291,
  29451,
  20000,
  2308,
  4573,
  27842,
  28289,
  22541,
  17599,
  19514,
  28702,
  5007,
  11369,
  22463,
  16195,
  16235,
  26894,
  33217,
  30780,
  30378,
  28185,
  7985,
  22489,
  16225,
  1771,
  18976,
  20054,
  28933,
  24625,
  29084,
  13069,
  5095,
  34519,
  20412,
  14408,
  22979,
  4822,
  32433,
  6790,
  22308,
  30691,
  12300,
  16823,
  15149,
  13078,
  27322,
  8099,
  32125,
  24154,
  26752,
  15837,
  9679,
  30600,
  27807,
  3606,
  20365,
  2263,
  25243,
  25546,
  19506,
  35474,
  28284,
  4575,
  18821,
  17392,
  14076,
  28591,
  34621,
  32616,
  16509,
  19395,
  15597,
  17920,
  24603,
  23191,
  22718,
  28661,
  27860,
  19068,
  191,
  32990,
  19132,
  4201,
  18527,
  17237,
  27806,
  17362,
  793,
  17111,
  4794,
  32703,
  7903,
  5752,
  28148,
  22642]}

### Step 2.2 - Creating the Ground Truth
Since we are splitting the data into train and validation, we will test the predictions we get with the training data in the validation set. To do this, we need to determine what is the ground truth (the actual outcomes) of the recommendations for the validation set. 

The main idea is to pick up a sorted array for each user on the songs with highest value in the validaton ratings matrix.

In [34]:
def get_y_true(R_val_, users_to_pred, n=100):
    """Get the ground truth (best recommendations) of the users in the validation set.
    
    Args:
        R_val_ (csr_matrix): Validation set ratings matrix.
        n (int): Number of top-n items.
        
    Returns:
        y_true_df (pd.DataFrame): DataFrame which returns the y_true items.
        
    """
    top_from_R_val = pd.DataFrame(np.negative(R_val_).toarray().argsort()[:, :n])
    y_true_df = pd.concat([users_to_pred, top_from_R_val], axis=1)
    y_true_df = y_true_df.set_index("users to recommend songs")
    return y_true_df


y_true_df = get_y_true(ratings_val, unique_users_training_data, n=100)
y_true_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,12420,1952,5896,27461,27462,27463,27464,27465,27466,27467,...,27388,27390,27391,27392,27393,27394,27395,27396,27389,27397
002511b392561fc1d426d875c386b356a6fc5702,5967,9243,10803,12524,10249,27462,27463,27464,27465,27466,...,27380,27388,27390,27391,27392,27393,27394,27395,27396,27389
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,12002,8794,3600,5113,6303,6909,27461,27462,27463,27464,...,27380,27388,27390,27391,27392,27393,27394,27395,27396,27389
003998bc33cddeba02428a43391c6716e523c8f7,10564,12295,0,27459,27460,27461,27462,27463,27464,27465,...,27378,27386,27388,27389,27390,27391,27392,27393,27394,27387
0042d2027dfa0340e31d2aa875c4be229730efb7,6099,0,27458,27459,27460,27461,27462,27463,27457,27464,...,27387,27388,27389,27390,27391,27392,27393,27386,27394,27376


In [35]:
# Create the dictionary with the ground truth.
y_true_dict = create_dict_preds(y_true_df)

## Evaluate
We will use the Mean Average Precision @ K to evaluate our predictions.

In [36]:
evaluate(y_true_dict, non_pers_dict)

5.251524099386366e-06

## Predict 
To submit your predictions, you just need to convert your recommendations to the formatwe have in the `example_output.csv` file.

In [37]:
# Join both dataframes with user_id's
all_test_users = pd.concat([test_users_in_data, test_users_not_in_data]).reset_index(drop=True)

In [38]:
non_pers_test_most_rated_df = convert_non_pers_recommendations_to_df(non_pers_most_rated, all_test_users)
print(non_pers_test_most_rated_df.shape)
non_pers_test_most_rated_df.head()

(1000, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
991411f0dca94f348c7bd3eae93b6e6c061605f1,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
323fbb28144eefa3eabfa22bd310dfb0713de80d,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
55c750f0951ca1021b26c0e758660bb8a2c49d3a,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
b458e3d697276a93aa6926caf1ff08e875933940,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642


In [39]:
def save_predictions(predictions: pd.DataFrame, output_path: str):
    """Save predictions to csv.
    
    Saves the predictions into a csv file with the format we need.
    We keep the index since it contains the user ids.
    
    Args:
        predictions (pd.DataFrame): DataFrame with user_id as index and ordered recommendations in the columns.
        output_path (str): Filepath for the predictions file.
    
    """
    predictions.to_csv(output_path, header=None)
    print(f"Saved to csv in '{output_path}'.")
    
    
save_predictions(non_pers_test_most_rated_df, os.path.join("data", "test_non_personalized_recommendations.csv"))

Saved to csv in 'data\test_non_personalized_recommendations.csv'.


In [40]:
# Filter the non-personalized recommendations for the users without training data and save
new_users_recommendations = non_pers_most_rated_df.iloc[index_users_not_in_data]
save_predictions(new_users_recommendations, os.path.join("data", "new_users_non_personalized.csv"))

Saved to csv in 'data\new_users_non_personalized.csv'.


## Step 3: Personalized


### Step 3.1 - Collaborative filtering


In [41]:
def make_user_similarities(R_):
    """Creates the user similarities matrix.
    
    Args:
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        user_similarities (csr_matrix): Matrix with user similarities.
    
    """
    return cosine_similarity(R_, dense_output=False)


user_similarities = make_user_similarities(ratings_train)
user_similarities

<7526x7526 sparse matrix of type '<class 'numpy.float64'>'
	with 1092971 stored elements in Compressed Sparse Row format>

In [42]:
def make_user_predictions_collab_filt(S: csr_matrix, R_: csr_matrix):
    """Predict using collaborative filtering.
    
    Args:
        S (csr_matrix): Similarities matrix (tipically using the cosine_similarity).
        R_ (csr_matrix): Ratings matrix.
        
    Returns:
        preds (csr_matrix): Predictions matrix.
    
    """
    weighted_sum = np.dot(S, R_)
    
    # We use the absolute value to support negative similarities.
    # In this particular example there are none.
    sum_of_weights = np.abs(S).sum(axis=1)
    
    preds = weighted_sum / sum_of_weights
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return csr_matrix(preds)
 

collab_filt_user_preds = make_user_predictions_collab_filt(user_similarities, ratings_train)
collab_filt_user_preds

C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\sparse\base.py:594: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


<7526x41194 sparse matrix of type '<class 'numpy.float64'>'
	with 9377700 stored elements in Compressed Sparse Row format>

In [43]:
def sparsity(matrix):
    """Calculates the sparsity of a matrix.
    
    Args:
        matrix (csr_matrix): Sparse matrix.
        
    Returns:
        sparsity_ (float): Sparsity percentage (between 0 and 1).
    
    """
    return 1 - matrix.nnz / (matrix.shape[0] * matrix.shape[1])


sparsity(collab_filt_user_preds)

0.9697518960697379

In [44]:
def get_most_rated_from_user_preds(user_preds_: csr_matrix, n: int) -> np.matrix:
    """Returns the n most rated items from the user predictions.
    
    Args:
        ratings (csr_matrix): A sparse ratings matrix
        n (int): The number of top-n items we should retrieve.
        
    Returns:
        most_rated (np.matrix): An array of the most rated items.
    
    """
    pred_ = np.negative(user_preds_).toarray()
    return pred_.argsort()[:, :n]


collab_filt_most_rated = get_most_rated_from_user_preds(collab_filt_user_preds, 100)
print(collab_filt_most_rated.shape)
collab_filt_most_rated

(7526, 100)


array([[17599, 19514, 31557, ..., 28856, 17486,  8929],
       [33212, 34271, 13847, ..., 30474, 16550, 25964],
       [ 8649, 26909, 29451, ...,  4637, 25532,  4314],
       ...,
       [ 5387, 28702, 20340, ...,  4193, 27025, 32196],
       [30897, 24612, 27307, ..., 32092, 27185,   741],
       [    0, 27457, 27458, ..., 27385, 27393, 27375]], dtype=int64)

In [45]:
def convert_pers_recommendations_to_df(pers_recs: np.array, users_to_pred: pd.DataFrame) -> pd.DataFrame:
    """Converts the personalized most rated to an DataFrame with the users and the recommendations.
    
    Args:
        pers_recs (np.array): Array of indices for the best personalized items to recommend.
        users_to_pred (pd.DataFrame): DataFrame containing the users which need recommendations.
        
    Returns:
        non_pers_most_rated_matrix (np.array): Two dimensional array of (n_users, top_n_items)
    
    """
    pers_df = pd.concat([users_to_pred, pd.DataFrame(pers_recs)], axis=1)
    pers_df = pers_df.set_index("users to recommend songs")
    
    return pers_df


collab_filt_most_rated_df = convert_pers_recommendations_to_df(collab_filt_most_rated, unique_users_training_data)
collab_filt_most_rated_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,17599,19514,31557,30668,14194,2291,10115,32616,8967,8341,...,18747,1022,8119,9679,30527,33317,30836,28856,17486,8929
002511b392561fc1d426d875c386b356a6fc5702,33212,34271,13847,23659,12842,4524,32419,7961,17854,734,...,30412,31106,13129,27307,7944,11094,29923,30474,16550,25964
002dfbc3c073b55a64a4abab34c0ca1f13897f1c,8649,26909,29451,8341,6291,11369,8925,28702,20000,6571,...,19514,6851,26752,30828,27806,27891,32433,4637,25532,4314
003998bc33cddeba02428a43391c6716e523c8f7,26658,5697,33162,27696,22584,3713,24154,35471,13456,1050,...,1191,8582,818,26072,3372,18639,11369,21107,14741,21821
0042d2027dfa0340e31d2aa875c4be229730efb7,3097,31331,25554,17303,22541,31042,16195,28577,14114,10766,...,20286,18322,13069,3204,19887,30176,27656,21275,882,35534


In [46]:
collab_filt_dict = create_dict_preds(collab_filt_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(collab_filt_dict.items())[0:1])

{'0011d5f4fb02ff276763d385c3f2ded2b00ad94a': [17599,
  19514,
  31557,
  30668,
  14194,
  2291,
  10115,
  32616,
  8967,
  8341,
  26658,
  15254,
  30780,
  26812,
  29084,
  9506,
  33145,
  25836,
  12716,
  28311,
  2341,
  28289,
  5289,
  26293,
  4822,
  4472,
  25131,
  35164,
  2781,
  1337,
  24154,
  11902,
  29867,
  22915,
  20725,
  33529,
  8388,
  16195,
  1189,
  2308,
  11560,
  3187,
  28933,
  22489,
  29513,
  25954,
  16225,
  31457,
  11369,
  22979,
  12767,
  22547,
  32433,
  8989,
  25530,
  21547,
  2854,
  5531,
  21487,
  29004,
  33290,
  28717,
  9856,
  33606,
  32501,
  34628,
  658,
  16235,
  25060,
  12595,
  11884,
  28179,
  15990,
  6291,
  3606,
  20000,
  6961,
  8333,
  15597,
  13228,
  30376,
  15498,
  13867,
  32272,
  7903,
  19068,
  1771,
  2263,
  5752,
  27860,
  18747,
  1022,
  8119,
  9679,
  30527,
  33317,
  30836,
  28856,
  17486,
  8929]}

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [47]:
evaluate(y_true_dict, collab_filt_dict)

0.030477454861760348

## Predict 
To submit your predictions, you just need to convert your recommendations to the format we have in the `example_output.csv` file.

In [48]:
# Filter the collaborative filtering most rated DataFrame  using the test_users_in_data mask.
#collab_filt_most_rated_in_data_df = collab_filt_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
# to_list() not available, use tolist() instead
collab_filt_most_rated_in_data_df = collab_filt_most_rated_df.loc[test_users_in_data.iloc[:, 0].tolist()]
print(collab_filt_most_rated_in_data_df.shape)
collab_filt_most_rated_in_data_df.head()

(700, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
56d985c92960b98ad76a48b10a062b0cd86795bf,2686,3097,16221,30422,11368,29087,35203,16195,31319,9064,...,27474,27473,27493,27471,27470,27482,0,27448,27452,27395
991411f0dca94f348c7bd3eae93b6e6c061605f1,21350,25246,4849,32542,6480,30930,24635,29434,18913,1208,...,27439,27461,27440,27442,27443,27444,27445,27446,27447,27441
323fbb28144eefa3eabfa22bd310dfb0713de80d,22227,22566,2783,28702,3318,4786,28111,34221,21679,7004,...,28495,20863,31814,15084,21234,9,25518,23613,24625,3325
55c750f0951ca1021b26c0e758660bb8a2c49d3a,2291,1189,2308,28702,28289,26053,35474,6291,32433,30780,...,7903,27777,1198,28495,21533,9953,17765,964,22956,10645
b458e3d697276a93aa6926caf1ff08e875933940,2291,29451,8341,6291,28702,33162,22541,1337,20000,1771,...,13456,22463,8848,4787,25546,30924,14818,23084,32469,3606


In [49]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

(300, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
users to recommend songs,,,,,,,,,,,,,,,,,,,,,
002511b392561fc1d426d875c386b356a6fc5702,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
003998bc33cddeba02428a43391c6716e523c8f7,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
005b1fab38cdeb9d5bb97debcf73b44050994a3e,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
005cc5d858319f13f88228f62341e5a4270f8e75,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642
006d3c79b9ed677280f8ddbc422d7b0fedd6d1fa,2291,1189,26658,8341,1337,6291,29451,20000,2308,4573,...,27806,17362,793,17111,4794,32703,7903,5752,28148,22642


In [50]:
collab_filt_most_rated_test_df = pd.concat([collab_filt_most_rated_in_data_df, new_users_recommendations])
print(collab_filt_most_rated_test_df.shape)

(1000, 100)


In [51]:
# Save the collaborative filtering recommendations 
save_predictions(collab_filt_most_rated_test_df, os.path.join("data", "collab_filt_recommendations.csv"))

Saved to csv in 'data\collab_filt_recommendations.csv'.


### Step 3.2 - Content-based recommendations

In [52]:
# If you had a FileNotFoundError, make sure you have unziped the `song_tag.zip` file.
def read_tags():
    """Import the song tags file.
    
    Returns:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        
    """
    path = os.path.join('data', 'song_tag.csv')
    data = pd.read_csv(path)
    return data


tags = read_tags()
print(tags.shape)
tags.head()

FileNotFoundError: File b'data\\song_tag.csv' does not exist

In [ ]:
def read_songs():
    """Import the songs file.
    
    Returns:
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
    
    """
    path = os.path.join('data', 'songs.txt')
    data = pd.read_csv(path, names=['song_id', 'song_index'], sep=' ')
    return data


songs= read_songs()
print(songs.shape)
songs.head()

In [ ]:
def merge_tags_songs(tags, songs, data):
    """Join the tags, songs and ratings data sources into a single frame).
    
    Args:
        tags (pd.DataFram): DataFrame with song_index, tag_index and value of the tag in the song.
        songs (pd.DataFrame): DataFrame containing the song_id and the corresponding song_index.
        
    Returns:
        tags_cleaned (pd.DataFrame): Matches the song_id to the tag_index with a corresponding value.
    
    """
    tags_cleaned = (tags.merge(songs, how='left', on='song_index')
                        .merge(data, how='right', on='song_id')
                   )[['song_id', 'tag_index', 'val']]
    return tags_cleaned

tags_cleaned = merge_tags_songs(tags, songs, data)
print(tags_cleaned.shape)
tags_cleaned.head()

In [ ]:
# Get the number of unique song_id's for which we have tags. 
tags_cleaned.song_id.nunique()

In [ ]:
def make_item_profiles(data):
    """Creates the item profiles matrix.
    
    Args:
        data (pd.DataFrame): DataFrame containing the (rows, columns, values) for the ratings matrix.
                             In this case, we have (song_id, tag_index, value).
    
    Returns:
        item_profiles (csr_matrix): Item profiles matrix. Items as rows, tags as columns.
    
    """
    items, item_pos = np.unique(data.iloc[:, 0].values, return_inverse=True)
    tags, tag_pos = np.unique(data.iloc[:, 1].values, return_inverse=True)
    values = data.iloc[:, 2].fillna(0).values
    
    shape = (len(items), len(tags))

    item_profiles_ = coo_matrix((values, (item_pos, tag_pos)), shape=shape).tocsr()
    return item_profiles_


item_profiles = make_item_profiles(tags_cleaned)
item_profiles

In [ ]:
def make_user_profiles(R_, item_profiles_):
    """Calculate the user profiles with the items.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        
    Returns:
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
        
    """
    return np.dot(R_, item_profiles_)


user_profiles = make_user_profiles(ratings_train, item_profiles)
user_profiles

In [ ]:
def make_user_predictions_content(R_, item_profiles_, user_profiles_):
    """Produces content-based predictions.
    
    Args:
        R_ (csr_matrix): Ratings matrix with shape (n_users, n_items).
        item_profiles_ (csr_matrix): Item profiles matrix with shape (n_items, n_tags).
        user_profiles (csr_matrix): User profiles considering the ratings and the item profiles, with shape (n_users, n_tags).
       
    Returns:
        preds (csr_matrix): Predictions of ratings using content-based recommendations.
    
    """
    
    # Since we are not looking at the values inbetween the cosine_similarity and excluding previously rated items,
    # we can define dense_output=False to speed up.
    preds = cosine_similarity(user_profiles_, item_profiles_, dense_output=False)
    
    # Exclude previously rated items.
    preds[R_.nonzero()] = 0
    
    return preds


content_user_preds = make_user_predictions_content(ratings_train, item_profiles, user_profiles)
content_user_preds

In [ ]:
content_most_rated = get_most_rated_from_user_preds(content_user_preds, 100)
print(content_most_rated.shape)
content_most_rated

In [ ]:
content_most_rated_df = convert_pers_recommendations_to_df(content_most_rated, unique_users_training_data)
print(content_most_rated_df.shape)
content_most_rated_df.head()

In [ ]:
content_dict = create_dict_preds(content_most_rated_df)
# Since dicts in python are not ordered, we need to HAMMER DOWN a way to print some values.
dict(list(content_dict.items())[0:2])

## Evaluate
We use again the Mean Average Precision @ K to evaluate our predictions.

In [ ]:
evaluate(y_true_dict, content_dict)

## Predict 
To submit your predictions, you just need to convert your recommendations to the formatwe have in the `example_output.csv` file.

In [ ]:
content_most_rated_in_data_df = content_most_rated_df.loc[test_users_in_data.iloc[:, 0].to_list()]
print(content_most_rated_in_data_df.shape)
content_most_rated_in_data_df.head()

In [ ]:
print(new_users_recommendations.shape)
new_users_recommendations.head()

In [ ]:
content_most_rated_test_df = pd.concat([content_most_rated_in_data_df, new_users_recommendations])
print(content_most_rated_test_df.shape)

In [ ]:
# Save the content-based filtering recommendations 
save_predictions(content_most_rated_test_df, os.path.join("data", "content_recommendations.csv"))